In [2]:
import pandas as pd
import os

In [23]:
def list_csv_files(directory):
    # List to hold csv file names
    csv_files = []
    
    # Iterate over files in the directory
    for filename in os.listdir(directory):
        # Check if the file is a CSV
        if filename.endswith('.csv'):
            csv_files.append(filename)
    
    return csv_files

folder_path = './phishing_data'
csv_files = list_csv_files(folder_path)

# List to hold dataframes
combined = []

# Specify the required columns
required_columns = ['sender', 'receiver', 'date', 'subject', 'body', 'urls', 'label']

# Process each file
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    
    # Read with TREC-specific parameters if needed
    if 'TREC' in file:
        df = pd.read_csv(file_path, on_bad_lines="warn", lineterminator="\n")
    else:
        df = pd.read_csv(file_path)
    
    # Check if the dataset contains the required 7 columns
    if set(required_columns).issubset(df.columns):
        # Only select the required columns
        df = df[required_columns]
        combined.append(df)

# Concatenate all the dataframes into one
combined_df = pd.concat(combined, ignore_index=True)

# Print the combined dataframe details
print(f"Combined DataFrame Shape: {combined_df.shape}")
print(f"Columns: {combined_df.columns}")


Combined DataFrame Shape: (184843, 7)
Columns: Index(['sender', 'receiver', 'date', 'subject', 'body', 'urls', 'label'], dtype='object')


In [24]:
# Remove rows with missing values in any of the columns
combined_df_cleaned = combined_df.dropna()

# Print the shape of the cleaned DataFrame to confirm removal of missing values
print(f"Combined DataFrame Shape after removing missing values: {combined_df_cleaned.shape}")


Combined DataFrame Shape after removing missing values: (174904, 7)


In [25]:
# Convert all columns to string except for 'date' and 'label'
combined_df = combined_df.astype({
    'sender': str,
    'receiver': str,
    'subject': str,
    'body': str,
    'urls': str,
    'label': int
})

# Convert 'date' column to datetime, using `errors='coerce'` to handle bad date strings
combined_df['date'] = pd.to_datetime(combined_df['date'], errors='coerce', utc=True)


From EDA, we came to realise that most of the URLs in the dataset was labelled 0 or 1. Hence, for the small percentage of rows that contain actual URL strings, we converted them to 1 and 0 too.

In [26]:
# Define a function to handle the URL column conversion only if necessary
def convert_urls(url_column_value):
    # If it's already 0 or 1, return as is
    if url_column_value == 0 or url_column_value == 1:
        return url_column_value
    # If it's an empty list "[]", convert to 0
    elif url_column_value == "[]":
        return 0
    # If it's a non-empty list (e.g., [http://example.com]), convert to 1
    elif url_column_value.startswith("[") and url_column_value.endswith("]"):
        return 1
    return 0  # Default fallback if not recognized

# Apply the conversion function to the 'urls' column
combined_df['urls'] = combined_df['urls'].apply(convert_urls)


In [27]:
# Convert all columns to string except for 'date' and 'label'
combined_df = combined_df.astype({
    'sender': str,
    'receiver': str,
    'subject': str,
    'body': str,
    'label': int
})

# Convert 'date' column to datetime, using `errors='coerce'` to handle bad date strings
combined_df['date'] = pd.to_datetime(combined_df['date'], errors='coerce', utc=True)

# Remove rows where 'date' column is NaT (invalid date)
combined_df = combined_df.dropna(subset=['date'])

In [ ]:
combined_df

### Feature Engineering

This is the feature engineering for the date column.

In [29]:

def is_business_hours(hour):
    return 9 <= hour < 17

def extract_date_features(df):
    features = pd.DataFrame()

    # Numerise the days of the week e.g. 0 = Monday, 6 = Sunday
    features['date_day_of_week'] = df['date'].dt.dayofweek 

    # Numerise the time of the day by the hour in 24 hour format
    features['date_hour_of_day'] = df['date'].dt.hour

    # Checks whether it is the weekend
    features['date_is_weekend'] = df['date'].dt.dayofweek >= 5

    # Checks whether it is during business hour
    features['date_is_business_hours'] = features['date_hour_of_day'].apply(is_business_hours)

    return features

phishing_date_features = extract_date_features(combined_df)
combined_df = pd.concat([combined_df.reset_index(drop=True), phishing_date_features.reset_index(drop=True)], axis=1)


Feature Engineering for subject

In [30]:
def extract_subject_features(df):
    features = pd.DataFrame()

    # Checks if the email is a reply to any previous mail
    features['subj_reply'] = df['subject'].apply(lambda x: x.lower().startswith("re:"))

    # Checks if the email is a forward from another mail
    features['subj_forward'] = df['subject'].apply(lambda x: x.lower().startswith("fwd:"))

    # Number of words in the subject
    features['subj_word_count'] = df['subject'].apply(lambda x: len(x.split()))

    # Number of characters in the subject
    features['subj_char_count'] = df['subject'].apply(lambda x: len(x))

    return features

# Extract subject features and add them to combined_df
subject_features = extract_subject_features(combined_df)
combined_df = pd.concat([combined_df.reset_index(drop=True), subject_features.reset_index(drop=True)], axis=1)



Feature Engineering for body

In [32]:
import pandas as pd
import re
from textblob import TextBlob
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Updated function to count the number of spam keywords in the body
def body_spam_count(body_content):
    spam_keywords = [
        "account", "access", "bank", "credit", "click", "identity", "inconvenience", "information", "limited",
        "log", "minutes", "password", "recently", "risk", "social", "security", "service", "suspended", "urgent", 
        "verify", "confirm", "locked", "unauthorized", "compromise", "refund", "billing", "update", "deactivate", 
        "subscription", "win", "prize", "offer", "discount", "reward", "free", "claim", "verify your account", 
        "reset password", "suspension", "fraud", "alert", "protection", "sign-in", "login", "activation", "urgent action"
    ]
    return sum(1 for word in body_content.lower().split() if word in spam_keywords)


# Feature engineering function
def extract_body_features(df):
    features = pd.DataFrame()
    
    # Basic text features
    features['body_char_count'] = df['body'].apply(len)
    features['body_word_count'] = df['body'].apply(lambda x: len(x.split()))
    features['body_distinct_word_count'] = df['body'].apply(lambda x: len(set(x.split())))
    features['body_average_word_length'] = features.apply(lambda row: row['body_char_count'] / row['body_word_count'] if row['body_word_count'] > 0 else 0, axis=1)

    # Uppercase and numeric characters
    features['body_uppercase_word_count'] = df['body'].apply(lambda x: sum(1 for word in x.split() if word.isupper()))
    features['body_numeric_char_count'] = df['body'].apply(lambda x: sum(c.isdigit() for c in x))

    # Punctuation features
    features['body_exclamation_count'] = df['body'].apply(lambda x: x.count('!'))
    features['body_question_count'] = df['body'].apply(lambda x: x.count('?'))
    features['body_special_char_count'] = df['body'].apply(lambda x: len(re.findall(r'[#$%&@]', x)))

    # Sentiment analysis
    features['body_sentiment_polarity'] = df['body'].apply(lambda x: TextBlob(x).sentiment.polarity)
    features['body_sentiment_subjectivity'] = df['body'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

    # URL analysis
    features['body_url_count'] = df['body'].apply(lambda x: len(re.findall(r'http[s]?://', x)))
    features['body_shortened_url_count'] = df['body'].apply(lambda x: len(re.findall(r'bit\.ly|t\.co|tinyurl', x)))

    # Spam count check
    features['body_spam_count'] = df['body'].apply(body_spam_count)
    
    return features

# Generate features
phishing_body_features = extract_body_features(combined_df)
combined_df = pd.concat([combined_df.reset_index(drop=True), phishing_body_features.reset_index(drop=True)], axis=1)

Feature Engineering for sender (Uses receiver too)

In [35]:
import pandas as pd
import re

def get_email_domain(email):
    """Helper function to extract the domain from an email address."""
    match = re.search(r'@([\w.-]+)', email)
    return match.group(1).lower() if match else None  # Normalize to lowercase

def send_diffSenderReceiver(send_address, receiver_addresses):
    """Check if sender domain is different from all receiver domains."""
    send_domain = get_email_domain(send_address)

    # Split receiver addresses and get their domains
    receiver_domains = [
        get_email_domain(receiver) for receiver in receiver_addresses.split(',')
    ]
    
    # Remove None values and normalize to lowercase
    receiver_domains = [domain.lower() for domain in receiver_domains if domain]

    # Check if any receiver domain is different from sender domain
    if send_domain and receiver_domains:
        return not all(domain == send_domain for domain in receiver_domains)  # Return True if any domain is different

    return False  # If sender or receiver domains are invalid, return False

# Functions to extract sender address based attributes
def extract_sender_features(df):
    features = pd.DataFrame()

    # Integer: number of words in sender address
    features['send_noWords'] = df['sender'].apply(lambda x: len(x.split()))

    # Integer: number of characters in sender address
    features['send_noCharacters'] = df['sender'].apply(lambda x: len(x))

    # Boolean: check if sender domain is different from all receiver domains
    features['send_diffSenderReceiver'] = df.apply(
        lambda row: send_diffSenderReceiver(row['sender'], row['receiver']), axis=1
    )

    return features

# Extract features and add to combined_df
features = extract_sender_features(combined_df)
combined_df = pd.concat([combined_df.reset_index(drop=True), features.reset_index(drop=True)], axis=1)



In [ ]:
combined_df